In [1]:
import json
import requests

URL_JOURNEY = '''https://gvh.hafas.de/hamm?requestId=undefined&hciMethod=StationBoard&hciVersion=
1.62&hciClientType=WEB&hciClientVersion=10107&aid=IKSEvZ1SsVdfIRSK&rnd=1730290996611'''

URL_LID = '''https://gvh.hafas.de/hamm?requestId=undefined&hciMethod=
LocMatch&hciVersion=1.62&hciClientType=WEB&hciClientVersion=10107&aid=IKSEvZ1SsVdfIRSK&rnd=1730291024259'''

DATA_JOURNEY = {"ver":"1.62",
                "lang":"deu",
                "auth":{
                    "type":"AID",
                    "aid":"IKSEvZ1SsVdfIRSK"},
                "client":
                    {
                        "id":"HAFAS",
                        "type":"WEB",
                        "name":"webapp",
                        "l":"vs_webapp",
                        "v":10107
                    },
                "formatted":'false',
                "svcReqL":[
                    {
                        "req":
                        {
                            "jnyFltrL":
                            [
                                {"type":"PROD",
                                "mode":"INC",
                                "value":831}
                                ],
                            "stbLoc":
                                {
                                    "name":"xxx",
                                    "lid":"de:03241:121",
                                    "extId":"xxx",
                                    "eteId":"xxx"
                                },
                            "type":"DEP",
                            "sort":"PT",
                            "maxJny":100},
                        "meth":"StationBoard",
                        "id":"1|5|"}
                    ]
                }

DATA_LID = {"ver":"1.62",
            "lang":"deu",
            "auth":
                {"type":"AID",
                 "aid":"IKSEvZ1SsVdfIRSK"},
            "client":{"id":"HAFAS",
                      "type":"WEB",
                      "name":"webapp",
                      "l":"vs_webapp",
                      "v":10107},
            "formatted":'false',
            "svcReqL":[
                {"req":
                    {"input":
                        {"field":"S",
                         "loc":{
                             "type":"S",
                             "dist":1000,
                             "name":"Hannover, Kröpcke"},
                         "maxLoc":7}
                        },
                    "meth":"LocMatch",
                    "id":"1|8|"}
                ]
            }


In [2]:
def get_lid(name: str = None,
            url: str = URL_LID,
            data: dict = DATA_LID) -> str:
    if name is not None:
        data["svcReqL"][0]["req"]["input"]["loc"]["name"] = name
        
    response = requests.post(url, json=data)
    contents = response.json()
    
    lid = contents["svcResL"][0]["res"]["match"]["locL"][0]["lid"]
    
    return lid


def get_journeys(name: str = None, 
                 url: str = URL_JOURNEY,
                 data: dict = DATA_JOURNEY,
                 num_journeys: int = 100) -> dict:
    
    if name is not None:
        data["svcReqL"][0]["req"]["stbLoc"]["lid"] = get_lid(name)
        
    data["svcReqL"][0]["req"]["maxJny"] = num_journeys

    response = requests.post(url, json=data)
    contents = response.json()
    journeys = contents["svcResL"][0]["res"]["jnyL"]
    
    return journeys

In [32]:
journeys = get_journeys(name="Hannover, Kröpcke", num_journeys=30)

In [33]:
for journey in journeys:
    train_name = journey["dirTxt"]
    
    if "dTimeR" in journey["stbStop"].keys():
        timeS = journey["stbStop"]["dTimeS"]
        timeR = journey["stbStop"]["dTimeR"]
        
        print(f"Train to {train_name} is delayed by {int(timeR) - int(timeS)} second[s].")
        
    else:
        print(f"No delay given for the train to {train_name}.")

Train to Hannover Fasanenkrug is delayed by 154 second[s].
Train to Hannover Hauptbahnhof is delayed by 100 second[s].
Train to Hannover/Zoo is delayed by 24 second[s].
Train to Empelde is delayed by 0 second[s].
Train to Garbsen is delayed by 0 second[s].
Train to Hannover/Misburg is delayed by 4100 second[s].
No delay given for the train to Langenhagen.
No delay given for the train to Laatzen.
Train to Hannover/Roderbruch is delayed by 300 second[s].
Train to Hannover/Wettbergen is delayed by 106 second[s].
Train to Hannover/Haltenhoffstraße is delayed by 0 second[s].
Train to Hannover Hauptbahnhof is delayed by 0 second[s].
Train to Hemmingen is delayed by 206 second[s].
Train to Messe/Ost(EXPO-Plaza) is delayed by 136 second[s].
Train to Hannover/Alte Heide is delayed by 0 second[s].
Train to Altwarmbüchen is delayed by 0 second[s].
Train to Stöcken is delayed by 100 second[s].
Train to Hannover Ahlem is delayed by 0 second[s].
Train to Hannover/Wettbergen is delayed by 206 second[

In [5]:
# roderbruch = [journey for journey in journeys if journey["dirTxt"] == "Hannover/Roderbruch"]

# for jrn in roderbruch:
#     print(jrn["stbStop"]["dTimeS"])

print(len(journeys))

10
